In [ ]:
# Sluice operation occuring only after Hmin
def TidalLagoonOP(Hstart, Hmin, oceanMin):
    ii = 0;
    Lagoon = [];
    Power = [];
    Qturbine = [];
    Qsluice = [];
    
    QActualT = 0;
    QActualS = 0;
    EnActualT = 0;
    
    lagoonElevation = 0; # Initial Lagoon Water level
    MinScale = 60;
    EnSum = 0;
    DeltaTime = 1; # 1 minute
    timeMin = np.linspace(0, len(oceanMin)-1, len(oceanMin)) # Time, by the minute
    top = "holding" # Starting Turbine operation
    for t in timeMin:
        oceanElevation = oceanMin[ii];
        Lagoon.append(lagoonElevation);
        head = oceanElevation - lagoonElevation; # -> Sinal positivo = fluxo em direção à lagoa
#         print("np.abs(head): ", np.abs(head), 'oceanElevation: ', oceanElevation, 'lagoonElevation: ', lagoonElevation)
        # QS = 0; QTc = 0; En = 0; -> OnEpisodeBegin
        QTo = QActualT; # Initial flow output from turbines, independent of operational mode
        QSo = QActualS; # Initial flow output from sluices, independent of operational mode
        ETo = EnActualT; # Initial Energy output from turbines, independent of operational mode

        if (top == "holding" and np.abs(head) >= Hstart):
            top = "powerGen"
        if (top == "powerGen" and np.abs(head) <= Hmin):
            top = "sluicing"
        if (top == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            top = "holding"

        # Calculated "target" flow-rates
        QS = sluices(g,rho,head,As,CD,top,MinScale) # Sluice control
        QIdl = idling(g,rho,head,At,nt,CDt,top,MinScale) # Turbine Idling control
        QTc, En = PowerGen(g, rho, nt, Sp, Dt, To, CE, Ca, head, nt, MinScale, top); # Turbine Power gen control
        # All ramp flow transitions simplified with momentum ramp function
        QActualT = -Zeta * (QTc + QIdl - QTo) + QTc + QIdl; # Eq: Qt+1 = -B*DeltaQ + Qc; Qc = Steady-state flow-rate, DeltaQ = Qc - Qt
        QActualS = -Zeta * (QS - QSo) + QS;
        EnActualT = -Zeta * (En - ETo) + En; # EnActualT is the positive energy accumulated in a minute for all Groups

        AlVar = np.interp(lagoonElevation, Al[:,0], Al[:,1])* 10**6 # Conversion from km2 to m2
        newLagoonElevation = (QActualT + QActualS) * DeltaTime / AlVar + lagoonElevation;
        EnSum += EnActualT; # Calculated Total Energy Sum
        lagoonElevation = newLagoonElevation # Update lagoon water level
        Power.append(EnActualT/MinScale)
        Qturbine.append(QActualT)
        Qsluice.append(QActualS)
        ii += 1

    return Lagoon, Power, Qturbine, Qsluice, EnSum

In [ ]:
# Sluice operation independent of turbine operation
def TidalLagoonOPB2(HtStart, HtMin, HsStart, oceanMin):
    ii = 0;
    Lagoon = [];
    Power = [];
    Qturbine = [];
    Qsluice = [];
    QActualT = 0;
    QActualS = 0;
    EnActualT = 0;
    lagoonElevation = 0; # Initial Lagoon Water level
    MinScale = 60;
    EnSum = 0;
    DeltaTime = 1; # 1 minute
    timeMin = np.linspace(0, len(oceanMin)-1, len(oceanMin)) # Time, by the minute
    top = "holding" # Starting Turbine operation
    sop = "holding"
    for t in timeMin:
        oceanElevation = oceanMin[ii];
        Lagoon.append(lagoonElevation);
        head = oceanElevation - lagoonElevation; # -> Sinal positivo = fluxo em direção à lagoa
        # QS = 0; QTc = 0; En = 0; -> OnEpisodeBegin
        QTo = QActualT; # Initial flow output from turbines, independent of operational mode
        QSo = QActualS; # Initial flow output from sluices, independent of operational mode
        ETo = EnActualT; # Initial Energy output from turbines, independent of operational mode
        
        if (top == "holding" and np.abs(head) >= HtStart):
            top = "powerGen"
        if (top == "powerGen" and np.abs(head) <= HtMin):
            top = "sluicing"
        if (top == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            top = "holding"
            sop = "holding"

        if ((top == "powerGen" or top == "sluicing") and np.abs(head) <= HsStart):
            sop = "sluicing"
        if (sop == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            sop = "holding"

        QS = sluices(g,rho,head,As,CD,sop,MinScale) # Sluice control
        QIdl = idling(g,rho,head,At,nt,CDt,top,MinScale) # Turbine Idling control
        QTc, En = PowerGen(g, rho, nt, Sp, Dt, To, CE, Ca, head, nt, MinScale, top); # Turbine Power gen control
        # All ramp flow transitions simplified with momentum ramp function
        QActualT = -Zeta * (QTc + QIdl - QTo) + QTc + QIdl; # Eq: Qt+1 = -B*DeltaQ + Qc; Qc = Steady-state flow-rate, DeltaQ = Qc - Qt
        QActualS = -Zeta * (QS - QSo) + QS;
        EnActualT = -Zeta * (En - ETo) + En; # EnActualT is the positive energy accumulated in a minute for all Groups

        AlVar = np.interp(lagoonElevation, Al[:,0], Al[:,1])* 10**6 # Conversion from km2 to m2
        newLagoonElevation = (QActualT + QActualS) * DeltaTime / AlVar + lagoonElevation;
        EnSum += EnActualT; # Calculated Total Energy Sum (Agent Reward)
        lagoonElevation = newLagoonElevation # Update lagoon water level
        Power.append(EnActualT/MinScale)
        Qturbine.append(QActualT)
        Qsluice.append(QActualS)
        ii += 1

    return Lagoon, Power, Qturbine, Qsluice, EnSum 

In [ ]:
# Sluice operation occuring only after Hmin
def TidalLagoonOPB3(Hstart, Hmin, timeMin, oceanMin, lagoonElevation, top, prevQandEn): # Initial Lagoon Water level as function input
    ii = 0;
    Lagoon = [];
    Power = [];
    Qturbine = [];
    Qsluice = [];
    QActualT = prevQandEn[0]; QActualS = prevQandEn[1];  EnActualT = prevQandEn[2];  EnSum = 0; 
    MinScale = 60;
    DeltaTime = 1; # 1 minute
    nextQandEn = [0, 0, 0];
    
    # Added:
    headp = 0 # Previous head
    derivHead = 0
    
    for t in timeMin:
        oceanElevation = oceanMin[ii];
        Lagoon.append(lagoonElevation);
        head = oceanElevation - lagoonElevation; # -> Sinal positivo = fluxo em direção à lagoa
        # Added
        derivHead = np.sign(np.abs(head) - headp); headp = np.abs(head)
        # QS = 0; QTc = 0; En = 0; -> OnEpisodeBegin
        QTo = QActualT; # Initial flow output from turbines, independent of operational mode
        QSo = QActualS; # Initial flow output from sluices, independent of operational mode
        ETo = EnActualT; # Initial Energy output from turbines, independent of operational mode
        
        if (top == "holding" and np.abs(head) >= Hstart) or (derivHead < 0 and np.abs(head) >= .8*Hstart):
          top = "powerGen"
        if (top == "powerGen" and np.abs(head) <= Hmin):
          top = "sluicing"
        if (top == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
          top = "holding"

        QS = sluices(g,rho,head,As,CD,top,MinScale) # Sluice control
        QIdl = idling(g,rho,head,At,nt,CDt,top,MinScale) # Turbine Idling control
        QTc, En = PowerGen(g, rho, nt, Sp, Dt, To, CE, Ca, head, nt, MinScale, top); # Turbine Power gen control

        # All ramp flow transitions simplified with momentum ramp function
        QActualT = -Zeta * (QTc + QIdl - QTo) + QTc + QIdl; # Eq: Qt+1 = -B*DeltaQ + Qc; Qc = Steady-state flow-rate, DeltaQ = Qc - Qt
        QActualS = -Zeta * (QS - QSo) + QS;
        EnActualT = -Zeta * (En - ETo) + En; # EnActualT is the positive (or negative) energy accumulated in a minute for all Groups

        AlVar = np.interp(lagoonElevation, Al[:,0], Al[:,1])* 10**6 # Conversion from km2 to m2
        newLagoonElevation = (QActualT + QActualS) * DeltaTime / AlVar + lagoonElevation;
        EnSum += EnActualT; # Calculated Total Energy Sum (Agent Reward)
        lagoonElevation = newLagoonElevation # Update lagoon water level
        Power.append(EnActualT/MinScale)
        Qturbine.append(QActualT)
        Qsluice.append(QActualS)
        ii += 1

    nextQandEn[0] = QActualT; nextQandEn[1] = QActualS; nextQandEn[2] = EnActualT;
    return Lagoon, Power, Qturbine, Qsluice, EnSum, top, nextQandEn

In [ ]:
def ehtB3(timeMin, oceanMinHT, initialLagoonElev, top, prevQandEn):
    resVec = [1, .5, .25, .125, .0625, .03125, 0.015625, 0.0078125] # Difference in cm between operational head tested
    iteration = 0
    MaxEn = 0
    HminBegin = 1
    HminEnd = 3
    HstartBegin = 1
    HstartEnd = 6
    BestConfig = [0,0]
    for res in resVec:
        # Brute Force optimization (Find best Hstart and Hmin for a whole month):
        if iteration != 0:
            HstartBegin = BestConfig[0] - res
            HstartEnd   = BestConfig[0] + res
            HminBegin   = BestConfig[1] - res
            HminEnd     = BestConfig[1] + res

        numberHmin = int(1 + (HminEnd - HminBegin)/res)
        numberHstart = int(1 + (HstartEnd - HstartBegin)/res)

        for Hmin in np.linspace(HminBegin, HminEnd, numberHmin):
            for Hstart in np.linspace(HstartBegin, HstartEnd, numberHstart):
                Lagoon, Power, Qturbine, Qsluice, EnOut, ntop, nextQandEn = TidalLagoonOPB3(Hstart, Hmin, timeMin, oceanMinHT, initialLagoonElev, top, prevQandEn)
                if (EnOut > MaxEn):
                    MaxEn = EnOut
                    BestConfig = [Hstart, Hmin]
                    NX = nextQandEn
                    NP = ntop
                    lake = Lagoon
                iteration += 1
    return MaxEn, lake, BestConfig, Power, Qturbine, Qsluice, NP, NX

In [ ]:
# Sluice operation occuring only after Hmin
def TidalLagoonOPB4(HtStart, HtMin, HsStart, timeMin, oceanMin, lagoonElevation, top, sop, prevQandEn): # Initial Lagoon Water level as function input
    ii = 0;
    Lagoon = [];
    Power = [];
    Qturbine = [];
    Qsluice = [];
    QActualT = prevQandEn[0]; QActualS = prevQandEn[1];  EnActualT = prevQandEn[2];  EnSum = 0;
    MinScale = 60;
    DeltaTime = 1; # 1 minute
    nextQandEn = [0, 0, 0]
    
    for t in timeMin:
        oceanElevation = oceanMin[ii];
        Lagoon.append(lagoonElevation);
        head = oceanElevation - lagoonElevation; # -> Sinal positivo = fluxo em direção à lagoa
        # QS = 0; QTc = 0; En = 0; -> OnEpisodeBegin
        QTo = QActualT; # Initial flow output from turbines, independent of operational mode
        QSo = QActualS; # Initial flow output from sluices, independent of operational mode
        ETo = EnActualT; # Initial Energy output from turbines, independent of operational mode

        if (top == "holding" and np.abs(head) >= HtStart):
            top = "powerGen"
        if (top == "powerGen" and np.abs(head) <= HtMin):
            top = "sluicing"
        if (top == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            top = "holding"
            sop = "holding"

        if ((top == "powerGen" or top == "sluicing") and np.abs(head) <= HsStart):
            sop = "sluicing"
        if (sop == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            sop = "holding"

        QS = sluices(g,rho,head,As,CD,sop,MinScale) # Sluice control
        QIdl = idling(g,rho,head,At,nt,CDt,top,MinScale) # Turbine Idling control
        QTc, En = PowerGen(g, rho, nt, Sp, Dt, To, CE, Ca, head, nt, MinScale, top); # Turbine Power gen control

        # All ramp flow transitions simplified with momentum ramp function
        QActualT = -Zeta * (QTc + QIdl - QTo) + QTc + QIdl; # Eq: Qt+1 = -B*DeltaQ + Qc; Qc = Steady-state flow-rate, DeltaQ = Qc - Qt
        QActualS = -Zeta * (QS - QSo) + QS;
        EnActualT = -Zeta * (En - ETo) + En; # EnActualT is the positive (or negative) energy accumulated in a minute for all Groups
        
        AlVar = np.interp(lagoonElevation, Al[:,0], Al[:,1])* 10**6 # Conversion from km2 to m2
        newLagoonElevation = (QActualT + QActualS) * DeltaTime / AlVar + lagoonElevation;
        EnSum += EnActualT; # Calculated Total Energy Sum (Agent Reward)
        lagoonElevation = newLagoonElevation # Update lagoon water level
        Power.append(EnActualT/MinScale)
        Qturbine.append(QActualT)
        Qsluice.append(QActualS)
        ii += 1
    nextQandEn[0] = QActualT; nextQandEn[1] = QActualS; nextQandEn[2] = EnActualT;
    return Lagoon, Power, Qturbine, Qsluice, EnSum, top, sop, nextQandEn

In [ ]:
# Sluice operation occuring only after Hmin
def TidalLagoonOPB4_New(HtStart, HtMin, HsStart, timeMin, oceanMin, lagoonElevation, top, sop, prevQandEn): # Initial Lagoon Water level as function input
    ii = 0;
    Lagoon = [];
    Power = [];
    Qturbine = [];
    Qsluice = [];
    QActualT = prevQandEn[0]; QActualS = prevQandEn[1];  EnActualT = prevQandEn[2];  EnSum = 0;
    MinScale = 60;
    DeltaTime = 1; # 1 minute
    nextQandEn = [0, 0, 0]
    
    # Added:
    headp = 0 # Previous head
    derivHead = 0
    
    for t in timeMin:
        oceanElevation = oceanMin[ii];
        Lagoon.append(lagoonElevation);
        head = oceanElevation - lagoonElevation; # -> Sinal positivo = fluxo em direção à lagoa
        # Added
        derivHead = np.sign(np.abs(head) - headp); headp = np.abs(head)
        # QS = 0; QTc = 0; En = 0; -> OnEpisodeBegin
        QTo = QActualT; # Initial flow output from turbines, independent of operational mode
        QSo = QActualS; # Initial flow output from sluices, independent of operational mode
        ETo = EnActualT; # Initial Energy output from turbines, independent of operational mode

        if (top == "holding" and np.abs(head) >= HtStart) or (derivHead < 0 and np.abs(head) >= .8*HtStart):
            top = "powerGen"
        if (top == "powerGen" and np.abs(head) <= HtMin):
            top = "sluicing"
        if (top == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            top = "holding"
            sop = "holding"

        if ((top == "powerGen" or top == "sluicing") and np.abs(head) <= HsStart):
            sop = "sluicing"
        if (sop == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            sop = "holding"

        QS = sluices(g,rho,head,As,CD,sop,MinScale) # Sluice control
        QIdl = idling(g,rho,head,At,nt,CDt,top,MinScale) # Turbine Idling control
        QTc, En = PowerGen(g, rho, nt, Sp, Dt, To, CE, Ca, head, nt, MinScale, top); # Turbine Power gen control

        # All ramp flow transitions simplified with momentum ramp function
        QActualT = -Zeta * (QTc + QIdl - QTo) + QTc + QIdl; # Eq: Qt+1 = -B*DeltaQ + Qc; Qc = Steady-state flow-rate, DeltaQ = Qc - Qt
        QActualS = -Zeta * (QS - QSo) + QS;
        EnActualT = -Zeta * (En - ETo) + En; # EnActualT is the positive (or negative) energy accumulated in a minute for all Groups
        
        AlVar = np.interp(lagoonElevation, Al[:,0], Al[:,1])* 10**6 # Conversion from km2 to m2
        newLagoonElevation = (QActualT + QActualS) * DeltaTime / AlVar + lagoonElevation;
        EnSum += EnActualT; # Calculated Total Energy Sum (Agent Reward)
        lagoonElevation = newLagoonElevation # Update lagoon water level
        Power.append(EnActualT/MinScale)
        Qturbine.append(QActualT)
        Qsluice.append(QActualS)
        ii += 1
    nextQandEn[0] = QActualT; nextQandEn[1] = QActualS; nextQandEn[2] = EnActualT;
    return Lagoon, Power, Qturbine, Qsluice, EnSum, top, sop, nextQandEn

In [ ]:
def ehtB4(timeMin, oceanMinHT, initialLagoonElev, top, sop, prevQandEn):
    resVec = [1, .5, .25, .125, .0625, .03125, 0.015625, 0.0078125] # Difference in cm between operational head tested
    iteration = 0
    MaxEn = 0
    HminBegin = 1
    HminEnd = 3
    HstartBegin = 1
    HstartEnd = 6
    HsStartBegin = 1
    HsStartEnd = 5
    BestConfig = [0,0,0]
    for res in resVec:
        # Brute Force optimization (Find best Hstart, Hmin and HsStart):
        if iteration != 0:

            HstartBegin    = BestConfig[0] - res
            HstartEnd     = BestConfig[0] + res
            HminBegin      = BestConfig[1] - res
            HminEnd       = BestConfig[1] + res
            HsStartBegin   = BestConfig[2] - res
            HsStartEnd    = BestConfig[2] + res

        numberHmin = int(1 + (HminEnd - HminBegin)/res)
        numberHstart = int(1 + (HstartEnd - HstartBegin)/res)
        numberHsStart = int(1 + (HsStartEnd - HsStartBegin)/res)

        for HtMin in np.linspace(HminBegin, HminEnd, numberHmin):
            for HtStart in np.linspace(HstartBegin, HstartEnd, numberHstart):
                for HsStart in np.linspace(HsStartBegin, HsStartEnd, numberHsStart):
                    Lagoon, Power, Qturbine, Qsluice, EnOut, ntop, nsop, nextQandEn = TidalLagoonOPB4(HtStart, HtMin, HsStart, timeMin, oceanMinHT, initialLagoonElev, top, sop, prevQandEn)
                    if (EnOut > MaxEn):
                        MaxEn = EnOut
                        BestConfig = [HtStart, HtMin, HsStart]
                        NX = nextQandEn
                        NP = ntop
                        NS = nsop
                        lake = Lagoon
                    iteration += 1
    return MaxEn, lake, BestConfig, Power, Qturbine, Qsluice, NP, NS, NX

In [ ]:
# start = timer() # BEGIN TIMER 
def etB3(timeMin1, oceanMinHT1, timeMin2, oceanMinHT2, initialLagoonElev, top, prevQandEn):
    resVec = [1, .5, .25, .125, .0625, .03125, 0.015625, 0.0078125] # Difference in cm between operational head tested
    iteration = 0
    MaxEn = 0
    MaxEn1 = 0
    HminBegin1 = 1
    HminEnd1 = 3
    HstartBegin1 = 1
    HstartEnd1 = 6
    HminBegin2 = 1
    HminEnd2 = 3
    HstartBegin2 = 1
    HstartEnd2 = 6
    BestConfig = [0, 0, 0, 0]
    for res in resVec:
        # Brute Force optimization (Find best Hstart and Hmin for a whole month):
        if iteration != 0:

            HstartBegin1 = BestConfig[0] - res
            HstartEnd1   = BestConfig[0] + res
            HminBegin1   = BestConfig[1] - res
            HminEnd1     = BestConfig[1] + res
            
            HstartBegin2 = BestConfig[2] - res
            HstartEnd2   = BestConfig[2] + res
            HminBegin2   = BestConfig[3] - res
            HminEnd2     = BestConfig[3] + res

        numberHmin1 = int(1 + (HminEnd1 - HminBegin1)/res)
        numberHstart1 = int(1 + (HstartEnd1 - HstartBegin1)/res)
        
        numberHmin2 = int(1 + (HminEnd2 - HminBegin2)/res)
        numberHstart2 = int(1 + (HstartEnd2 - HstartBegin2)/res)

        for Hmin1 in np.linspace(HminBegin1, HminEnd1, numberHmin1):
            for Hmin2 in np.linspace(HminBegin2, HminEnd2, numberHmin2):
                for Hstart1 in np.linspace(HstartBegin1, HstartEnd1, numberHstart1):
                    for Hstart2 in np.linspace(HstartBegin2, HstartEnd2, numberHstart2):
                        Lagoon1, Power1, Qturbine1, Qsluice1, EnOut1, ntop1, nextQandEn1 = TidalLagoonOPB3(Hstart1, Hmin1, timeMin1, oceanMinHT1, initialLagoonElev, top, prevQandEn)
                        Lagoon2, Power2, Qturbine2, Qsluice2, EnOut2, ntop2, nextQandEn2 = TidalLagoonOPB3(Hstart2, Hmin2, timeMin2, oceanMinHT2, Lagoon1[-1], ntop1, nextQandEn1)
                        if (EnOut1+EnOut2 > MaxEn): # Greatest sum of energy for both consecutive HT
                            MaxEn = EnOut1+EnOut2 # Store value of first half tide output
                            MaxEn1 = EnOut1
                            BestConfig = [Hstart1, Hmin1, Hstart2, Hmin2] # Store config value of first HT
                            NX = nextQandEn1
                            NP = ntop1
                        iteration = 1
    return MaxEn1, Lagoon1, BestConfig[0:2], Power1, Qturbine1, Qsluice1, NP, NX

USED 6TH BASELINE:

In [ ]:
def TidalLagoonOPB4FULLTIDE(params, *args):
    HtStart1, HtMin1, HsStart1 = params[0], params[1], params[2]
    HtStart2, HtMin2, HsStart2 = params[3], params[4], params[5]
    
    timeMin1, oceanMin1, lagoonElevation1 = args[0]['timeMin1'], args[0]['oceanMin1'], args[0]['lagoonElevation1']
    top1, sop1 = args[0]['top1'], args[0]['sop1']
    timeMin2, oceanMin2 = args[0]['timeMin2'], args[0]['oceanMin2']

    Lagoon1, Power1, Qturbine1, Qsluice1, EnSum1, ntop1, nsop1 = TidalLagoonOPB4N(HtStart1, HtMin1, HsStart1, timeMin1, oceanMin1, lagoonElevation1, top1, sop1)
    _, _, _, _, EnSum2, _, _               = TidalLagoonOPB4N(HtStart2, HtMin2, HsStart2, timeMin2, oceanMin2, Lagoon1[-1], ntop1, nsop1)
    
    return EnSum1 + EnSum2

In [ ]:
def TidalLagoonOPB4N(HtStart, HtMin, HsStart, timeMin, oceanMin, lagoonElevation, top, sop): # Initial Lagoon Water level as function input
    ii = 0;
    Lagoon = [];
    Power = [];
    Qturbine = [];
    Qsluice = [];
    QActualT = 0
    QActualS = 0
    EnActualT = 0
    MinScale = 60;
    EnSum = 0;
    DeltaTime = 1; # 1 minute
    
    for t in timeMin:
        oceanElevation = oceanMin[ii];
        Lagoon.append(lagoonElevation);
        head = oceanElevation - lagoonElevation; # -> Sinal positivo = fluxo em direção à lagoa
        # QS = 0; QTc = 0; En = 0; -> OnEpisodeBegin
        QTo = QActualT; # Initial flow output from turbines, independent of operational mode
        QSo = QActualS; # Initial flow output from sluices, independent of operational mode
        ETo = EnActualT; # Initial Energy output from turbines, independent of operational mode

        if (top == "holding" and np.abs(head) >= HtStart):
            top = "powerGen"
        if (top == "powerGen" and np.abs(head) <= HtMin):
            top = "sluicing"
        if (top == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            top = "holding"
            sop = "holding"

        if ((top == "powerGen" or top == "sluicing") and np.abs(head) <= HsStart):
            sop = "sluicing"
        if (sop == "sluicing" and np.abs(head) <= MaxDelta*1.1): # tolerance for considering oceanWl = lagoon Wl
            sop = "holding"

        QS = sluices(g,rho,head,As,CD,sop,MinScale) # Sluice control
        QIdl = idling(g,rho,head,At,nt,CDt,top,MinScale) # Turbine Idling control
        QTc, En = PowerGen(g, rho, nt, Sp, Dt, To, CE, Ca, head, nt, MinScale, top); # Turbine Power gen control

        # All ramp flow transitions simplified with momentum ramp function
        QActualT = -B * (QTc + QIdl - QTo) + QTc + QIdl; # Eq: Qt+1 = -B*DeltaQ + Qc; Qc = Steady-state flow-rate, DeltaQ = Qc - Qt
        QActualS = -B * (QS - QSo) + QS;
        EnActualT = -B * (En - ETo) + En; # EnActualT is the positive (or negative) energy accumulated in a minute for all Groups

        AlVar = np.interp(lagoonElevation, Al[:,0], Al[:,1])* 10**6 # Conversion from km2 to m2
        newLagoonElevation = (QActualT + QActualS) * DeltaTime / AlVar + lagoonElevation;
        EnSum += EnActualT; # Calculated Total Energy Sum (Agent Reward)
        lagoonElevation = newLagoonElevation # Update lagoon water level
        Power.append(EnActualT/MinScale)
        Qturbine.append(QActualT)
        Qsluice.append(QActualS)
        ii += 1

    return Lagoon, Power, Qturbine, Qsluice, -EnSum, top, sop 
#     return -EnSum

In [ ]:
# start = timer() # BEGIN TIMER 
def etB4(timeMin1, oceanMinHT1, timeMin2, oceanMinHT2, initialLagoonElev, top, sop):
    resVec = [.5, .25, .125, .0625, .03125, 0.015625, 0.0078125] # Difference in cm between operational head tested
    iteration = 0
    MaxEn = 0
    MaxEn1 = 0
    # First HT Optimizaion:
    additional = {'timeMin1': timeMin1, 'oceanMin1':oceanMinHT1, 'lagoonElevation1':initialLagoonElev, 
                  'top1':top, 'sop1':sop, 'timeMin2': timeMin2, 'oceanMin2':oceanMinHT2}
    x0 = [4, 2, 2, 4, 2, 2] # Initial Guess
#     minimizer_kwargs = {"method": "L-BFGS-B", "args": (additional,),"bounds": ((2, 6), (1, 3), (1, 6), (2, 6), (1, 3), (1, 6))}
    minimizer_kwargs = {"method": "COBYLA", "args": (additional,)}
    res = \
        opt.basinhopping(TidalLagoonOPB4FULLTIDE, 
                         x0=x0, 
                         minimizer_kwargs=minimizer_kwargs,
                         T=1,  # The “temperature” parameter
                         stepsize=1)  # Initial step size

    HtStart1, HtMin1, HsStart1 = res.x[0], res.x[1], res.x[2]
    HtStart2, HtMin2, HsStart2 = res.x[3], res.x[4], res.x[5]
    BestConfig = [HtStart1, HtMin1, HsStart1, HtStart2, HtMin2, HsStart2]
    
    Lagoon1, Power, Qturbine, Qsluice, EnSum, ntop1, nsop1 = TidalLagoonOPB4N(HtStart1, HtMin1, HsStart1, timeMin1, oceanMinHT1, initialLagoonElev, top, sop)
    MaxEn1 = -EnSum
    print('BestConfig: ', BestConfig)

    return MaxEn1, Lagoon1, BestConfig[0:3], Power, Qturbine, Qsluice, ntop1, nsop1